In [1]:
import os 
import warnings 
warnings.filterwarnings('ignore')
from glob import glob 
from tqdm import tqdm 
import pickle 

import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler

import torch 
import torch.nn as nn  
from torch import Tensor
from torch.utils.data import Dataset,DataLoader 

from src.Preprocess import df_preprocess,Normalize,split_window,data_aug

1. Normalize 
2. x,y split 
3. train - valid - test split 
4. augmentation 

In [5]:
raw_data = {'train':train_df,
            'test':test_df
           }
with open('./Data/raw_data.pickle', 'wb') as f:
    pickle.dump(raw_data, f, pickle.HIGHEST_PROTOCOL)

In [2]:
def data_load(mode='pickle'):
    if mode == 'pickle':
        with open('./Data/raw_data.pickle', 'rb') as f:
            data = pickle.load(f)
        train_df = data['train']
        test_df = data['test']
    else:
        train_df = pd.read_excel('./Data/SWaT_Dataset_Normal_v0.xlsx')
        test_df = pd.read_excel('./Data/SWaT_Dataset_Attack_v0.xlsx')
    return train_df,test_df 

#train_df,test_df = data_load('raw')
train_df,test_df = data_load()
#Column, dtype, time sort 
train = df_preprocess(train_df)
test = df_preprocess(test_df)

#Normalize
train,test  = Normalize(train,test)() # -1 ~ 1 Scaling 
#window x,y split 
train_x,train_y = split_window(train) # -140 ~ x_i ~ +180 
test_x,test_y = split_window(test)    


#augmentation 
res_train_x,res_train_y = data_aug(train_x,train_y)
res_test_x,res_test_y = data_aug(test_x,test_y)
#transpose
res_train_x = res_train_x.transpose(0,2,1)
res_test_x= res_test_x.transpose(0,2,1)




data = {'train':{'x':res_train_x,
          'y':res_train_y},
 ''
 'test':{'x':res_test_x,
         'y':res_test_y}
}

100%|██████████| 44946/44946 [01:10<00:00, 641.35it/s]


In [3]:
with open('./Data/pre_data.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)